In [1]:
# Notebook: 1-Automated-Labeler.ipynb (Final Version with Original Colab Logic)

import os
import cv2
from PIL import Image
from transformers import pipeline
import torch
from moviepy.video.io.VideoFileClip import VideoFileClip
import gc

# --- CONFIGURATION ---
# IMPORTANT: UPDATE THIS PATH to your project folder on your computer.
BASE_DIR = "D:/Distracted_Driving_Detection_Project"

RAW_VIDEOS_DIR = os.path.join(BASE_DIR, "raw_videos")
OUTPUT_CLIPS_DIR = BASE_DIR

# --- USING THE ORIGINAL, MORE EFFECTIVE LABELS FROM THE COLAB SCRIPT ---
CANDIDATE_LABELS = [
    "a person driving normally looking forward",
    "a person looking to the left side",
    "a person looking to the right side",
    "a person looking down at a phone"
]
FOLDER_LABELS = ["Normal_Driving", "Looking_Left", "Looking_Right", "Using_Phone"]

# --- LOAD AI MODEL ---
print("🧠 Loading the Zero-Shot Classification model...")
device = 0 if torch.cuda.is_available() else -1
if device == 0:
    print("GPU detected! Model will run on the GPU for faster processing.")
else:
    print("No GPU detected. Model will run on the CPU (this will be slow).")

classifier = pipeline(
    "zero-shot-image-classification",
    model="openai/clip-vit-large-patch14",
    device=device
)
print("✅ AI Model loaded successfully.")

# --- PROCESSING LOGIC ---
FRAME_INTERVAL = 15
MIN_CLIP_DURATION_SECONDS = 2.0

video_files = [f for f in os.listdir(RAW_VIDEOS_DIR) if f.endswith(('.mp4', '.avi', '.mov'))]
print(f"\nFound {len(video_files)} videos to process.")

for video_filename in video_files:
    video_path = os.path.join(RAW_VIDEOS_DIR, video_filename)
    print(f"\n▶️ Processing video: {video_filename}")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"  ❌ Error: Could not open video file.")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS) if cap.get(cv2.CAP_PROP_FPS) > 0 else 30
    min_frames_for_clip = int(fps * MIN_CLIP_DURATION_SECONDS)
    frame_predictions = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret: break
        if frame_count % FRAME_INTERVAL == 0:
            try:
                pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                outputs = classifier(pil_image, candidate_labels=CANDIDATE_LABELS)
                top_prediction = outputs[0]['label']
                frame_predictions.append(top_prediction)
                print(f"  Frame {frame_count}: Classified as '{top_prediction}'")
            except Exception as e:
                print(f"  ⚠️ Error classifying frame {frame_count}: {e}")
        frame_count += 1
    cap.release()
    
    if not frame_predictions:
        print("  ❌ No frames were classified for this video.")
        continue

    print(f"  ✅ Finished classifying frames. Now grouping actions and cutting clips...")
    current_action = frame_predictions[0]
    start_frame = 0
    clip_counter = 0

    for i in range(1, len(frame_predictions)):
        if frame_predictions[i] != current_action:
            end_frame = i * FRAME_INTERVAL
            segment_frame_count = end_frame - start_frame
            if segment_frame_count >= min_frames_for_clip:
                start_time = start_frame / fps
                end_time = end_frame / fps
                action_index = CANDIDATE_LABELS.index(current_action)
                folder_name = FOLDER_LABELS[action_index]
                output_folder = os.path.join(OUTPUT_CLIPS_DIR, folder_name)
                os.makedirs(output_folder, exist_ok=True)
                output_filename = f"{os.path.splitext(video_filename)[0]}_{folder_name}_{clip_counter}.mp4"
                output_path = os.path.join(output_folder, output_filename)
                print(f"    🎬 Cutting clip for '{folder_name}' ({start_time:.2f}s to {end_time:.2f}s)")
                try:
                    with VideoFileClip(video_path) as video_clip:
                        # Ensure end_time doesn't exceed video duration
                        safe_end_time = min(end_time, video_clip.duration)
                        if safe_end_time > start_time:
                            new_clip = video_clip.subclip(start_time, safe_end_time)
                            new_clip.write_videofile(output_path, codec="libx264", audio_codec="aac", logger=None)
                            clip_counter += 1
                except Exception as e:
                    print(f"      ⚠️ Error cutting clip: {e}")
            current_action = frame_predictions[i]
            start_frame = i * FRAME_INTERVAL

    # This block correctly processes the final clip
    start_time = start_frame / fps
    try:
        with VideoFileClip(video_path) as video_clip:
            end_time = video_clip.duration
            if (end_time - start_time) >= MIN_CLIP_DURATION_SECONDS:
                action_index = CANDIDATE_LABELS.index(current_action)
                folder_name = FOLDER_LABELS[action_index]
                output_folder = os.path.join(OUTPUT_CLIPS_DIR, folder_name)
                os.makedirs(output_folder, exist_ok=True)
                output_filename = f"{os.path.splitext(video_filename)[0]}_{folder_name}_{clip_counter}.mp4"
                output_path = os.path.join(output_folder, output_filename)
                print(f"    🎬 Cutting final clip for '{folder_name}' ({start_time:.2f}s to {end_time:.2f}s)")
                new_clip = video_clip.subclip(start_time, end_time)
                new_clip.write_videofile(output_path, codec="libx264", audio_codec="aac", logger=None)
                clip_counter += 1
    except Exception as e:
        print(f"      ⚠️ Error cutting final clip: {e}")

# Clean up memory
del classifier
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n\n🎉 All videos processed.")

🧠 Loading the Zero-Shot Classification model...
No GPU detected. Model will run on the CPU (this will be slow).



Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


✅ AI Model loaded successfully.

Found 31 videos to process.

▶️ Processing video: VID20250610110247.mp4
  Frame 0: Classified as 'a person looking to the right side'
  Frame 15: Classified as 'a person driving normally looking forward'
  Frame 30: Classified as 'a person looking to the right side'
  Frame 45: Classified as 'a person looking to the right side'
  Frame 60: Classified as 'a person looking to the right side'
  Frame 75: Classified as 'a person looking to the right side'
  Frame 90: Classified as 'a person looking to the right side'
  Frame 105: Classified as 'a person looking to the right side'
  Frame 120: Classified as 'a person looking to the right side'
  Frame 135: Classified as 'a person looking to the right side'
  Frame 150: Classified as 'a person looking to the right side'
  Frame 165: Classified as 'a person looking to the right side'
  Frame 180: Classified as 'a person looking to the right side'
  Frame 195: Classified as 'a person looking to the right side'
